In [1]:
# Dependencies
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Define League ID and year
league_id = 730129

In [3]:
# Define year range
years = [2018, 2019, 2020, 2021]

In [4]:
# Define the URL with our parameters
url = f'https://fantasy.espn.com/apis/v3/games/ffl/leagueHistory/{league_id}/?seasonId='

# Initialize empty list to hold each year's JSON response
performance = []

for year in years:
    
    # Pull team data from the URL 
    performance_response = requests.get(url + str(year), params={"view": "mTeam"})

    # Transform the response into a JSON file
    performance_json = performance_response.json()
    
    # Append each year to performance list
    performance.append(performance_json)

In [5]:
# Initialize empty lists to hold ids, W %
ids = []

win_percentage = []

In [6]:
# Loop through the JSON and populate ids and W % lists
for i in range(len(performance)):
        for j in range(0, 10):
            try:
                ids.append(performance[i][0]['teams'][j]['id'])
                win_percentage.append(performance[i][0]['teams'][j]['record']['overall']['percentage'])
            except:
                pass

In [7]:
# Create dictionary to hold performance data
data = {'Team ID' : ids, 'Win Percentage' : win_percentage}

# Load the dictionary into the DataFrame
performance_df = pd.DataFrame(data)

In [8]:
performance_df

,Team ID,Win Percentage
0,1,0.833333
1,2,0.333333
2,3,0.666667
3,4,0.500000
4,5,0.166667
5,6,0.416667
6,7,0.500000
7,8,0.583333
8,1,0.416667
9,2,0.583333


In [9]:
draft = []

for year in years:
    # Reuse the URL variable to call the draft view
    draft_response = requests.get(url, params={"view" : "mDraftDetail"})

    # Convert the response to a JSON
    draft_json = draft_response.json()
    
    # Append each year's draft data to the list
    draft.append(draft_json)

In [10]:
# Initialize the empty lists for ids and draft position
ids = []

draft_position = []

In [11]:
# Loop through the JSON and populate ids and draft position
for j in range(len(draft)):
    for i in range(0,10):
        try:
            ids.append(draft[0][j]['draftDetail']['picks'][i]['teamId'])
            draft_position.append(draft[0][j]['draftDetail']['picks'][i]['id'])
        except:
            pass

In [12]:
# SPECIFIC TO MY LEAGUE
# We did an 8 man league the first year, removing the 9th and 10th  from the lists 

del ids[8:10]
del draft_position[8:10]

In [13]:
# Create a dictionary to hold the draft data
data = {'Team ID' : ids, 'Draft Position' : draft_position}

# Load the dictionary into a pandas DataFrame
draft_df = pd.DataFrame(data)

In [14]:
draft_df

,Team ID,Draft Position
0,3,1
1,4,2
2,1,3
3,2,4
4,8,5
5,7,6
6,6,7
7,5,8
8,1,1
9,8,2


In [15]:
df = pd.merge(draft_df, performance_df, how='left')

In [16]:
df

,Team ID,Draft Position,Win Percentage
0,3,1,0.666667
1,3,1,0.583333
2,3,1,0.666667
3,3,1,0.461538
4,4,2,0.500000
...,...,...,...
113,3,9,0.461538
114,2,10,0.333333
115,2,10,0.583333
116,2,10,0.500000


In [17]:
# Drop the Team ID column
df = df[['Draft Position', 'Win Percentage']]

In [19]:
# Average win percentage by draft position
win_percentage_by_draft_position = df.groupby(df['Draft Position']).mean()

# Sort by Win Percentage
win_percentage_by_draft_position.sort_values(by=['Win Percentage'], ascending=False)

,Win Percentage
Draft Position,
9,0.599715
1,0.572344
2,0.545673
4,0.540293
3,0.536630
5,0.529304
7,0.515734
10,0.469373
6,0.445157
